In [1]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
from nltk.tokenize import sent_tokenize
import networkx as nx
import re

In [42]:
class summerizer:
    
    def __init__(self, *arg, **kwarg):
        pass
    
    def read_clean(self, data):
        """Read, clean and split the given file"""
        article = sent_tokenize(data)
        sentences = []    
        for sentence in article:
            sentence = re.sub('[/(){}\[\]\|@,;!٪×،*ـ+؟؛""]', " ", sentence)
            sentence = re.sub('[\\n]', "", sentence)
            sentence = re.sub('[!٬٫﷼٪×*)(ـ+}|؛؟<>‌ ÷؛«» "" -]', " ", sentence).split(" ")
            sentences.append(sentence)
        #     sentences.pop()
        return sentences
    
    
    def sentence_similarity(self, sent1, sent2, stop_words=None):
        if stop_words is None:
            stop_words = []
        all_words = list(set(sent1+sent2))
        # Building zero valued vectors with length : len(all_words)
        vector1 = [0]*len(all_words)
        vector2 = [0]*len(all_words)
        # Building vector for first(sent1) and second(sent2) sentence
        for w in sent1:
            if w in stopwords.words("persian"):
                continue
            vector1[all_words.index(w)] += 1

        for w in sent2:
            if w in stopwords.words("persian"):
                continue
            vector2[all_words.index(w)] += 1
        return 1 - cosine_distance(vector1, vector2)
    
    
    def build_similarity_matrix(self, sentences, stop_words):
        """Building similarity_matrix based on cosine_distance of two sentenece"""
        similarity_matrix = np.zeros((len(sentences), len(sentences))) # Create an empty similarity matrix
        for idx1 in range(len(sentences)):
            for idx2 in range(len(sentences)):
                if idx1 == idx2:
                    continue
                similarity_matrix[idx1][idx2] = self.sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
        return similarity_matrix
    
    def generate_summary(self, path, top_n=3):
        stop_words = stopwords.words("persian")
        summarize_text = []
        # read, clean and split text file
        sentences = self.read_clean(path)
        # Generate similarity matrix across the sentences
        sentence_similarity_matrix = self.build_similarity_matrix(sentences, stop_words)
        # biuld a graph for numpy array
        sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
        # rank nodes(sentences) in graph based on structure of the incoming links
        scores = nx.pagerank(sentence_similarity_graph)
        # Sort the rank and pick top sentences
        ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
        # print("Indexes of top ranked_sentence order are: ", ranked_sentence)
        
        for i in range(top_n):
            if len(sentences) <= top_n:
                print("متن انتخابی کوتاه است")
                break
            else:
                summarize_text.append(" ".join(ranked_sentence[i][1]))

        return print(". ".join(summarize_text))

In [43]:
s = """مؤمنه خاتون همسر بهاءالدین ولد و مادر جلال‌الدین محمد مولاناست. گور او در قرامان / لارنده کشف شده، بنابراین باید بین سال‌های ۶۲۶–۶۱۹/۱۲۲۹–۱۲۲۲ از دنیا رفته باشد.[۴۴]

در سال‌های بعد از ۶۱۷ قمری یعنی اواسط دهه ۱۲۲۰ میلادی بهاءالدین ولد و خانواده‌اش که جلال‌الدین محمد بلخی (مولوی) نیز در آن بود به آناتولی مرکزی، روم رسیدند. لقب رومی جلال‌الدین از اینجاست. آنان مدتی در لارنده / کرمن کنونی توقف کردند. مردم آن سامان هنوز هم به دیدن مسجد کوچکی که به افتخار او – مؤمنه خاتون – ساخته شده می‌روند.[۴۵]

کرمن، پایتخت سلجوقیان روم، در حدود یکصد کیلومتری جنوب خاوری قونیه واقع است، علاءالدین کیقباد که عالمان و عارفان سراسر دنیا را گرد خود جمع کرده بود، بهاءالدین ولد پدر مولوی را به این شهر فراخواند.[۴۶] قونیه شهری بود که بهاءالدین ولد و خانواده اش پس از چهار سال اقامت در لارنده در سال ۶۲۶ یا ۶۲۷ قمری = ۱۲۲۸ میلادی در آنجا سکنی گزیدند.[۴۷]

ظاهراً مولانا و برادرش علاءالدین – که بعدها مولوی یکی از پسرانش را به نام او نامگذاری کرد – از دختر قاضی مشرف بوده‌اند و بهاءولد زن یا زنان دیگری داشته و احتمالاً از آن‌ها نیز صاحب فرزندانی بوده‌است. بهاءولد در معارف خود از دو زن یاد کرده‌است.[۴۸]

بعضی مدعی شده‌اند که خانواده پدری بهاءالدین از احفاد ابوبکر، خلیفه اول اسلام هستند، این ادعا چه حقیقت داشته باشد و چه نداشته باشد، دربارهٔ پیشینه قومی این خانواده هیچ اطلاع مسلمی در دست نیست. نیز گفته شده که همسر بهاءالدین از خاندان خوارزمشاهیان بوده‌است که در ولایات خاوری حدود سال ۳–۴۷۲ قمری حکومت خود را پایه‌گذاری کردند، ولی این داستان را هم می‌توان جعلی دانست و رد کرد. خوارزمشاه در سال ۳–۶۰۲ قمری بلخ موطن جلال‌الدین را که در تصرف غوریان بود تسخیر کرد.[۴۹]

اسلاف مولانا - چنان‌که فرزند او سلطان ولد نیز بدین معنا اشارت دارد از تباری عظیم و بزرگ بودند. البته شاید روایتی که در انتساب سلطان العلما به خلیفه اول یعنی ابوبکر بن ابی قحافه به افواه افتاده و رواج یافته، از آن باشد که نام جد مادری وی «ابوبکر» بوده‌است، و بعدها نام شمس‌الائمه ابوبکر محمد، با نام ابوبکر – نخستین خلیفه راشدین درآمیخته باشد.[۵۰] خاندان بهاءولد هم نسبت صدیقی و ابوبکری داشت و هم نسبت علوی ادعا می‌کرد.[۵۱"""

In [44]:
Summ = summerizer()

In [46]:
%time
Summ.generate_summary(s, 4)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
 ۴۴ در سال های بعد از ۶۱۷ قمری یعنی اواسط دهه ۱۲۲۰ میلادی بهاءالدین ولد و خانواده اش که جلال الدین محمد بلخی  مولوی  نیز در آن بود به آناتولی مرکزی  روم رسیدند..  ۴۵ کرمن  پایتخت سلجوقیان روم  در حدود یکصد کیلومتری جنوب خاوری قونیه واقع است  علاءالدین کیقباد که عالمان و عارفان سراسر دنیا را گرد خود جمع کرده بود  بهاءالدین ولد پدر مولوی را به این شهر فراخواند.. گور او در قرامان   لارنده کشف شده  بنابراین باید بین سال های ۶۲۶–۶۱۹ ۱۲۲۹–۱۲۲۲ از دنیا رفته باشد..  ۴۸ بعضی مدعی شده اند که خانواده پدری بهاءالدین از احفاد ابوبکر  خلیفه اول اسلام هستند  این ادعا چه حقیقت داشته باشد و چه نداشته باشد  دربارهٔ پیشینه قومی این خانواده هیچ اطلاع مسلمی در دست نیست.
